In [1]:
import pandas as pd

# 데이터 불러오기
ks200_data = pd.read_csv('../main/ks200.csv', encoding='utf-8')

# '날짜'를 datetime 형식으로 변환 및 정렬
ks200_data['날짜'] = pd.to_datetime(ks200_data['날짜'], format='%Y-%m-%d')
ks200_data = ks200_data.sort_values(by='날짜')

# 연-월 정보 추출
ks200_data['year_month'] = ks200_data['날짜'].dt.to_period('M')

# 각 월의 마지막 두 거래일 추출
last_two_trading_days = ks200_data.groupby('year_month').tail(2)

# 'base_date'와 'before_last'를 저장할 새로운 데이터프레임 생성
trading_days_v2 = pd.DataFrame()

# 'before_last'와 'base_date' 추출 및 저장
trading_days_v2['before_last'] = last_two_trading_days.groupby('year_month').nth(0)['날짜'].values
trading_days_v2['base_date'] = last_two_trading_days.groupby('year_month').nth(1)['날짜'].values

# 'base_date'를 인덱스로 설정
trading_days_v2.set_index('base_date', inplace=True)

# 결과를 새로운 CSV 파일로 저장
trading_days_v2.to_csv('new.csv', encoding='utf-8-sig')

# 결과의 처음 몇 행을 출력
trading_days_v2.head()


,before_last
base_date,
2011-04-29,2011-04-28
2011-05-31,2011-05-27
2011-06-30,2011-06-29
2011-07-29,2011-07-28
2011-08-31,2011-08-30


In [5]:
import yfinance as yf
import pandas as pd

# yfinance 라이브러리를 사용하여 KOSPI 지수 데이터를 가져옵니다.
kospi = yf.download('^KS11', start='2011-04-01', end='2023-10-01', progress=False)

# 닫는 가격만 사용합니다.
kospi = kospi['Close']

# 'trading_days_v2.csv' 데이터를 불러옵니다.
trading_days_v2 = pd.read_csv('new.csv', parse_dates=['base_date', 'before_last'])
trading_days_v2.set_index('base_date', inplace=True)

# 각 월별 KOSPI 지수를 계산합니다.
trading_days_v2['KOSPI'] = kospi.loc[trading_days_v2.index]
# m_return을 계산합니다. 
# (이번 달 KOSPI 지수 / 지난 달 KOSPI 지수) - 1
trading_days_v2['m_return'] = (kospi.loc[trading_days_v2.index].values / kospi.loc[trading_days_v2['before_last'].values].values) - 1
# 결과를 확인합니다.
print(trading_days_v2.head())

# 결과를 csv 파일로 저장합니다.
trading_days_v2.to_csv('2.csv', encoding='utf-8-sig')


           before_last        KOSPI  m_return
base_date                                    
2011-04-29  2011-04-28  2192.360107 -0.007241
2011-05-31  2011-05-27  2142.469971  0.020107
2011-06-30  2011-06-29  2100.689941  0.002994
2011-07-29  2011-07-28  2133.209961 -0.010502
2011-08-31  2011-08-30  1880.109985  0.019682
